<a href="https://colab.research.google.com/github/hesandism/rock-paper-scissor/blob/main/rock_paper_scissor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


In [20]:
MOVE_TO_INT = {"R": 0, "P": 1, "S": 2}
INT_TO_MOVE = {0: "R", 1: "P", 2: "S"}

def counter_move(move):
    return {"R": "P", "P": "S", "S": "R"}[move]


In [21]:
def build_model():
    model = Sequential([
        Dense(16, activation="relu", input_shape=(9,)),
        Dense(16, activation="relu"),
        Dense(3, activation="softmax")
    ])
    model.compile(
        optimizer=Adam(learning_rate=0.01),
        loss="categorical_crossentropy"
    )
    return model

model = build_model()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [47]:
def player(prev_play, opponent_history=[]):
    # Reset state at start of a new match
    if prev_play == "":
        opponent_history.clear()
        return random.choice(["R", "P", "S"])

    opponent_history.append(prev_play)

    # Early randomness to avoid exploitation
    if len(opponent_history) < 5:
        return random.choice(["R", "P", "S"])

    # Online training
    if len(opponent_history) >= 4:
        X, y = [], []

        for i in range(len(opponent_history) - 3):
            seq = opponent_history[i:i+3]
            input_vec = np.zeros(9)

            for j, m in enumerate(seq):
                input_vec[j * 3 + MOVE_TO_INT[m]] = 1

            X.append(input_vec)
            y.append(MOVE_TO_INT[opponent_history[i+3]])

        X = np.array(X)
        y = tf.keras.utils.to_categorical(y, 3)

        if len(opponent_history) % 20 == 0:
          model.fit(X, y, epochs=30, verbose=0)



    # Predict next opponent move
    last_three = opponent_history[-3:]
    input_vec = np.zeros(9)

    for i, m in enumerate(last_three):
        input_vec[i * 3 + MOVE_TO_INT[m]] = 1

    prediction = model.predict(input_vec.reshape(1, -1), verbose=0)
    predicted_move = INT_TO_MOVE[np.argmax(prediction)]

    # Play counter move
    return counter_move(predicted_move)


In [48]:
def quincy(prev_play, counter=[0]):
    counter[0] += 1
    return ["R", "P", "S"][counter[0] % 3]

def abbey(prev_play, opponent_history=[]):
    if prev_play:
        opponent_history.append(prev_play)
    return random.choice(["R", "P", "S"])

def kris(prev_play):
    if prev_play == "":
        return "R"
    return counter_move(prev_play)

def mrugesh(prev_play, opponent_history=[]):
    if prev_play:
        opponent_history.append(prev_play)
    if len(opponent_history) < 2:
        return random.choice(["R", "P", "S"])
    most_common = max(set(opponent_history), key=opponent_history.count)
    return counter_move(most_common)


In [49]:
def play(player1, player2, num_games, verbose=False):
    p1_prev = ""
    p2_prev = ""
    p1_score = 0
    p2_score = 0

    for i in range(num_games):
        p1 = player1(p2_prev)
        p2 = player2(p1_prev)

        if p1 == counter_move(p2):


            p1_score += 1
        elif p2 == counter_move(p1):
            p2_score += 1

        if verbose:
            print(f"Game {i+1}: {player1}={p1} {player2}={p2}")

        p1_prev = p1
        p2_prev = p2

    win_rate = p1_score / num_games
    print(f"Player1 win rate: {win_rate:.2%}")
    return win_rate


In [50]:
print("Against Quincy")
play(player, quincy, 500)

print("\nAgainst Abbey")
play(player, abbey, 500)

print("\nAgainst Kris")
play(player, kris, 500)

print("\nAgainst Mrugesh")
play(player, mrugesh, 500)


Against Quincy
Player1 win rate: 99.40%

Against Abbey
Player1 win rate: 29.60%

Against Kris
Player1 win rate: 44.80%

Against Mrugesh
Player1 win rate: 72.80%


0.728